In [1]:
import pandas as pd
import sqlite3
import numpy as np

# Read sqlite query results into a pandas DataFrame
con = sqlite3.connect("C:/Users/user/Downloads/MBZUAI/AI701/AI project/database.sqlite")
df_match = pd.read_sql_query("SELECT * from Match", con)
df_country = pd.read_sql_query("SELECT * from Country", con)
df_league = pd.read_sql_query("SELECT * from League", con)
df_player = pd.read_sql_query("SELECT * from Player", con)
df_team = pd.read_sql_query("SELECT * from Team", con)
df_team_attributes = pd.read_sql_query("SELECT * from Team_Attributes", con)
df_player_attributes = pd.read_sql_query("SELECT * from Player_Attributes", con)

con.close()

In [2]:
def update_season(date):   
    date = pd.to_datetime(date)
    season = ''
    if pd.to_datetime('2008-07-01') <= date <= pd.to_datetime('2009-06-30'):
        season = '2008/2009'
        
    elif pd.to_datetime('2009-07-01') <= date <= pd.to_datetime('2010-06-30'):
        season = '2009/2010'

    elif pd.to_datetime('2010-07-01') <= date <= pd.to_datetime('2011-06-30'):
        season = '2010/2011'
    
    elif pd.to_datetime('2011-07-01') <= date <= pd.to_datetime('2012-06-30'):
        season = '2011/2012'
    
    elif pd.to_datetime('2012-07-01') <= date <= pd.to_datetime('2013-06-30'):
        season = '2012/2013'
    
    elif pd.to_datetime('2013-07-01') <= date <= pd.to_datetime('2014-06-30'):
        season = '2013/2014'
    
    elif pd.to_datetime('2014-07-01') <= date <= pd.to_datetime('2015-06-30'):
        season = '2014/2015'
    
    elif pd.to_datetime('2015-07-01') <= date <= pd.to_datetime('2016-06-30'):
        season = '2015/2016' 
    
    return season

In [3]:
df_team_attributes['season_new'] = df_team_attributes['date'].map(update_season)

In [4]:
df_match = df_match.merge(df_team_attributes, left_on=['home_team_api_id','season'], right_on = ['team_api_id','season_new'], how='left')
df_match = df_match.merge(df_team_attributes, left_on=['away_team_api_id','season'], right_on = ['team_api_id','season_new'], how='left')

In [5]:
df_player_attributes['season_new'] = df_player_attributes['date'].map(update_season)

In [6]:
df_match['label'] = (df_match['home_team_goal']-df_match['away_team_goal']).apply(np.sign)

In [7]:
print (df_match.shape)
df_match = df_match[df_match.columns.drop(list(df_match.filter(regex='Class')))]
print (df_match.shape)

df_match.fillna(value=0,inplace=True)


df_match.drop(['season_new_x'],inplace=True,axis=1)
df_match.drop(['season_new_y'],inplace=True,axis=1)
df_match.drop(['season'],inplace=True,axis=1)

df_match.drop(['home_team_goal','away_team_goal','goal','date','shoton','shotoff','foulcommit','card','cross','corner','possession'],inplace=True,axis=1)
df_match.drop(['date_x','date_y'],inplace=True,axis=1)


(25979, 168)
(25979, 144)


In [8]:
X = df_match.iloc[:,:-1]
y = df_match.iloc[:,-1]

In [13]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2,random_state=0)

In [14]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

model = DecisionTreeClassifier(random_state=0)

model.fit(X_train,y_train)

y_pred = model.predict(X_test)

accuracy_score(y_pred,y_test)

0.42282525019245576